### notebook purpose

- compute energy coefficient of k-space
- see if there's a correlation w ssim output

In [7]:
import os, sys
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import matplotlib.pyplot as plt
import torch

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.evaluate import calc_metrics_imgs, plot_row_qdess, scale_0_1
from utils.data_io import get_mtr_ids_and, load_imgs, load_h5_qdess
from utils.transform import reshape_adj_channels_to_complex_vals, \
                            reshape_complex_vals_to_adj_channels

In [30]:
def load_gt_ksp(file_id_list):
    
    NUM_SAMPS = len(file_id_list)
    ksp_set = np.empty((NUM_SAMPS, 2, 512, 160))
    
    for file_id in file_id_list:

        # load, concat both echo slices
        ksp = load_h5_qdess(file_id)
        idx_kx = ksp.shape[0] // 2 # want central slice in kx (axial) b/c we undersample in (ky,kz)
        ksp_echo1 = ksp[:,:,:,0,:].permute(3,0,1,2)[:, idx_kx, :, :]
        ksp_echo2 = ksp[:,:,:,1,:].permute(3,0,1,2)[:, idx_kx, :, :]
        ksp_orig = torch.cat((ksp_echo1, ksp_echo2), 0)

        print(ksp_orig.shape)
        
    return 

# TODO

### load ksp data

- load gt
    - if slow, may want to save middle slice as proxy
- load dc (running over weekend)
- get matching shapes, scale range

In [ ]:
load_gt_ksp(mtr_id_list)

### load images, but need in ksp

In [18]:
accel = 4
path_base = '/bmrNAS/people/dvv/out_qdess/accel_{}x/'.format(accel)

path_gt = path_base + 'gt/'
path_bl = path_base + 'baseline/'
# path_new = path_base + 'mask_dual/'

path_mask = '/home/vanveen/ConvDecoder/masks/mask_poisson_disc_{}x.npy'.format(accel)

mask = abs(np.load(path_mask)).astype(np.uint8)

In [23]:
mtr_id_list = ['005', '006', '030'] #get_mtr_ids_and(path_bl, path_gt)

imgs_gt = load_imgs(mtr_id_list, path=path_gt)
imgs_bl = load_imgs(mtr_id_list, path=path_bl)

metrics_bl = calc_metrics_imgs(imgs_gt, imgs_bl)
# mu_bl = np.around(np.mean(metrics_bl, 0), 4)

In [15]:
metrics_bl.shape, imgs_bl.shape

((34, 2, 4), (34, 2, 512, 160))